# Data Set

In [ ]:
from random import random
import numpy as np

In [ ]:
import os
import requests
from zipfile import ZipFile
import numpy as np
from scipy.interpolate import interp2d


class SRTM3API:

    SRTM3_URL = 'https://dds.cr.usgs.gov/srtm/version2_1/SRTM3'
    SRTM3API_DIR = os.environ['HOME'] + '/.srtm_api'
    SRTM3_RES = 1201
    SRTM3_REGIONS = [
        # TODO: Complete this list
        {'lon': (-15, 180), 'lat': (35, 61), 'code': 'Eurasia'},
        {'lon': (60, 180), 'lat': (-10, 35), 'code': 'Eurasia'},
        {'lon': (-26, 60), 'lat': (-35, 35), 'code': 'Africa'},
    ]

    def __init__(self, area=[]):
        """
        Downloading SRTM3 data from usgs.gov and creating a mesh

        Parameter
        area: (bl_lat, bl_on, tr_lat, tr_lon), bl: bottom left, tr: top right
        """

        # Create srtm_api config directory
        if 'HOME' not in os.environ:
            return

        if not os.path.exists(self.SRTM3API_DIR):
            os.makedirs(self.SRTM3API_DIR)

        # Create region directories inside srtm_api directory
        for region in self.SRTM3_REGIONS:
            dirname = self.SRTM3API_DIR + '/' + region['code']
            if not os.path.exists(dirname):
                os.makedirs(dirname)

        xpixels = (int(area[3] - area[1]) + 1) * self.SRTM3_RES
        ypixels = (int(area[2] - area[0]) + 1) * self.SRTM3_RES

        self.data = {
            'bl': {'lat': int(area[0]), 'lon': int(area[1])},
            'tr': {'lat': int(area[2]) + 1, 'lon': int(area[3]) + 1},
            'resolution': (xpixels, ypixels),
            'mesh': np.zeros((xpixels, ypixels), dtype=np.dtype('>i2'))
        }

        self._download_area()
        self._load_hgt_data()

    def elevation(self, lon, lat, interpolate=True):
        """
        Return the elevation of a given (lon, lat) by interpolating STRM3 data

        Parameter
        lon: longitude
        lat: latitude
        interpolate: If True, uses interpolation to find the elevation
        """

        dlon = float(lon) - self.data['bl']['lon']
        dlat = float(lat) - self.data['bl']['lat']

        x = dlon * self.SRTM3_RES
        y = dlat * self.SRTM3_RES

        i = int(x)
        j = int(y)

        max_i = self.data['resolution'][0]
        max_j = self.data['resolution'][1]

        if interpolate:
            if i - 1.5 < 0 or i + 3.5 > max_i or j - 1.5 < 0 or j + 3.5 > max_j:
                raise RuntimeError('Out of range!', lon, lat)

            xi, yi = np.meshgrid(
                np.arange(i - 1.5, i + 3.5, 1.0),
                np.arange(j - 1.5, j + 3.5, 1.0))
            f = interp2d(
                xi, yi, self.data['mesh'][i-2:i+3, j-2:j+3], kind='cubic')

            return f(x, y)[0]
        else:
            if not 0 <= i < max_i or not 0 <= j < max_j:
                raise RuntimeError('Out of range!', lon, lat)

            return self.data['mesh'][i, j]

    def _download_area(self):
        """
        Downloading SRTM3 hgt files
        """
        for lon in range(self.data['bl']['lon'], self.data['tr']['lon']):
            for lat in range(self.data['bl']['lat'], self.data['tr']['lat']):
                mdata = self._get_metadata(lon, lat)

                if os.path.exists(mdata['file']['path']):
                    continue

                hgt = requests.get(mdata['url'])
                open(mdata['zip']['path'], 'wb').write(hgt.content)

                hgtzip = ZipFile(mdata['zip']['path'], 'r')
                hgtzip.extractall(mdata['directory']['path'])
                hgtzip.close()

    def _get_region_code(self, lon, lat):
        """
        Getting the region name based on (lon, lat)
        """
        for region in self.SRTM3_REGIONS:
            if lon >= region['lon'][0] and lon <= region['lon'][1]:
                if lat >= region['lat'][0] and lat <= region['lat'][1]:
                    return region['code']

        raise RuntimeError('Region not found!', lon, lat)

    def _get_metadata(self, lon, lat):
        regionname = self._get_region_code(lon, lat)

        dirpath = self.SRTM3API_DIR + '/' + regionname

        lon_tag = "%s%03d" % ('E' if int(lon) > 0 else 'W', abs(int(lon)))
        lat_tag = "%s%02d" % ('N' if int(lat) > 0 else 'S', abs(int(lat)))
        filename = lat_tag + lon_tag + '.hgt'

        return {
            'region': regionname,
            'directory': {
                'name': regionname,
                'path': dirpath
            },
            'file': {
                'name': filename,
                'path': dirpath + '/' + filename,
            },
            'zip': {
                'name': filename + '.zip',
                'path': dirpath + '/' + filename + '.zip',
            },
            'url': self.SRTM3_URL + '/' + regionname + '/' + filename + '.zip',
        }

    def _load_hgt_data(self):
        """
        Loading downloaded .hgt files into self.data
        """
        for lon in range(self.data['bl']['lon'], self.data['tr']['lon']):
            for lat in range(self.data['bl']['lat'], self.data['tr']['lat']):
                mdata = self._get_metadata(lon, lat)

                i = (lon - self.data['bl']['lon']) * self.SRTM3_RES
                i_end = i + self.SRTM3_RES

                j = (lat - self.data['bl']['lat']) * self.SRTM3_RES
                j_end = j + self.SRTM3_RES

                # We need to flip the hgt matrix along y-dir since it is
                # arranged from west to east and then **north to south**
                self.data['mesh'][i:i_end, j:j_end] = np.flip(
                    np.fromfile(
                        mdata['file']['path'],
                        np.dtype('>i2'),
                        self.SRTM3_RES*self.SRTM3_RES
                    ).reshape((self.SRTM3_RES, self.SRTM3_RES)), 1  # y-dir
                )

In [ ]:
bl = {'lat': 47.37645735, 'lon': 8.4330368}
tr = {'lat': 47.42727336, 'lon': 8.55671103}

area = [bl['lat'], bl['lon'], tr['lat'], tr['lon']]

srtm = SRTM3API(area)

In [ ]:
N = 500

nodes = np.zeros((N, 1), dtype=[('lat', np.float64), ('lon', np.float64), ('elev', np.float64)])

for node in nodes:
    lat = bl['lat'] + random() * (tr['lat'] - bl['lat'])
    lon = bl['lon'] + random() * (tr['lon'] - bl['lon'])
    elev = srtm.elevation(lon, lat)
    node['lat'] = lat
    node['lon'] = lon
    node['elev'] = elev
    
print(nodes)
print(np.min(nodes['elev']), np.max(nodes['elev']))

## Data Structure

### Triangles:
- A list of thress node ids,
- Parent triangle(s)
- Child triangle(s)
- Neighbor triangle(s)

In [ ]:
DS = {
    'ntriangles': 0,
    'triangles': {}
}

def new_triangle(node_ids, parents=[], children=[], neighbors=[]):
    DS['ntriangles'] += 1
    triangle_id = DS['ntriangles']
    
    return {
        'id': triangle_id,
        'nodes': node_ids,
        'parents': parents,
        'children': children,
        'neighbors': neighbors,
    }

def remove_triangle(triangle_id):
    # del DS['triangles'][triangle_id]
    # delete neighbors
    pass

## Root triangle

- Finding the most upper right node

In [ ]:
# Finding the most upper right node
upper_right_node_id = 0 # id of the most upper right node

In [ ]:
# Adding the root triangle to triagnles
DS['triangles'][1] = new_triangle(node_ids=[upper_right_node_id, None, None], parents=None, children=[], neighbors=[])

## Locate
Locating the triangle which contains a given node

In [ ]:
# Locating the triangle which contains a given node

def locate(node):
    pntr = 1
    while DS['triangles'][ptr]['children']:
        # check which children contains the node
        children = [DS['triangles'][key] for key in DS['triangles'][ptr]['children']]
        for triangle in children:
            # if node in side the child:
                # triangle_key = 
            # elif node is on the edge of another triangle:
                # Create new trianles...
            # else:
                # error
        
        pntr = triangle_key
    
    return DS['triangles'][triangle_key] 

In [ ]:
# Dealunay algorithm
for node_id, node in enumerate(nodes):
    if node_id == upper_right_node_id:
        continue
        
    

In [ ]:
# Todo locating the position of new node


In [ ]:
def clockwise(triangle):
    """
    triangle : list of 3 elements
    """
    if_clockwise = True
    
    a = triangle[0]
    b = triangle[1]
    c = triangle[2]
    
    vect = np.array([[a[0], a[1], 1], [b[0], b[1], 1], [c[0], c[1], 1]])
    
    if np.linalg.det(vect) > 0:
        if_clockwise = False
        
    return if_clockwise

In [ ]:
def in_circle(triangle, d):
    
    """
    triangle : list of 3 elements
    d : the new node we added to triangulation
    """
    
    in_circle = False
    
    a = triangle[0]
    b = triangle[1]
    c = triangle[2]
    
    if clockwise(a, b, c):
        array = np.array([[a[0], a[1], (a[0]^2) + (a[1]^2), 1], [b[0], b[1], (b[0]^2) + (b[1]^2), 1], 
              [c[0], c[1], (c[0]^2) + (c[1]^2), 1], [d[0], d[1], (d[0]^2) + (d[1]^2), 1]])
    else:
        array = np.array([[a[0], a[1], (a[0]^2) + (a[1]^2), 1], [c[0], c[1], (c[0]^2) + (c[1]^2), 1]
                          [b[0], b[1], (b[0]^2) + (b[1]^2), 1], [d[0], d[1], (d[0]^2) + (d[1]^2), 1]])
        
    
    det = np.linalg.det(array) 
    if det > 0:
        in_circle = True
        
    return in_circle

In [ ]:
def adj_delta((p_i, p_j)):
    
    triangles = []
    
    for delta in triangulation:
        
        if (p_i, p_j) in delta or (p_j, p_i) in delta:
            triangles.append(delta)
            
    return triangles

In [ ]:
# p0_y = np.max(nodes['lon'])

# choices = [node for node in nodes if node['lon'] == p0_y]

# p0 = np.max([choices[i]['lat'] for i in range(len(choices)) ] )

    